<a href="https://colab.research.google.com/github/pratamastan/bangkit-capstone0143/blob/main/NLP_project1_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Fetching and train-test split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
# import pathlib
# from os import getcwd

Training data is taken from Quora Questions Pairs dataset (training set only) https://www.kaggle.com/c/quora-question-pairs/overview translated to Indonesian with Google Translate

In [ ]:
data1=pd.read_csv("/content/drive/MyDrive/Quora question pair dataset/train complete indo.csv")

In [ ]:
data1=data1.dropna()

In [ ]:
data1.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,Apa panduan langkah demi langkah untuk berinve...,Apa panduan langkah demi langkah untuk berinve...,0
1,1,3,4,Bagaimana kisah Berlian Kohinoor (Koh-i-Noor)?,Apa yang akan terjadi jika pemerintah India me...,0
2,2,5,6,Bagaimana cara meningkatkan kecepatan koneksi ...,Bagaimana kecepatan Internet dapat ditingkatka...,0
3,3,7,8,Mengapa saya secara mental sangat kesepian? Ba...,Temukan sisanya ketika [math] 23 ^ {24} [/ mat...,0
4,4,9,10,Manakah yang larut dalam air seperti gula pasi...,Ikan apa yang akan bertahan hidup di air asin?,0


In [ ]:
data1.question1.str.len().max()

717

In [ ]:
data1.question2.str.len().max()

1395

Train-test split 80% 20%

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data1[["question1", "question2"]], data1["is_duplicate"], test_size=0.2, random_state=42)

Transform the dataframe to NumPy array as TensorFlow takes NumPy arrays

In [ ]:
xt=X_train.to_numpy()
yt=y_train.to_numpy()

xte=X_test.to_numpy()
yte=y_test.to_numpy()

Q1_train = X_train["question1"].to_numpy()
Q2_train = X_train["question2"].to_numpy()
Q1_test = X_test["question1"].to_numpy()
Q2_test = X_test["question2"].to_numpy()

In [ ]:
Q1_train

array(['Apa sajakah cerita pendeknya?',
       'Apa jawaban terbaik untuk pertanyaan HR, "Anda berasal dari latar belakang ECE, jadi mengapa Anda ingin bergabung dengan perusahaan IT"?',
       'Bagaimana cara membulatkan 1,44 ke persen terdekat?', ...,
       'Siapa yang saat ini memenangkan pemilihan presiden?',
       'Apa sajakah film Telugu / Tamil / Hindi yang pemeran utamanya menjadi mafia atau tidak?',
       'Berapa peringkat yang dipotong untuk perguruan tinggi Kedokteran Rohtak dalam kuota negara bagian?'],
      dtype=object)

In [ ]:
data1["question1"][0]

'Apa panduan langkah demi langkah untuk berinvestasi di pasar saham di India?'

In [ ]:
data1[["question1","question2"]]

,question1,question2
0,Apa panduan langkah demi langkah untuk berinve...,Apa panduan langkah demi langkah untuk berinve...
1,Bagaimana kisah Berlian Kohinoor (Koh-i-Noor)?,Apa yang akan terjadi jika pemerintah India me...
2,Bagaimana cara meningkatkan kecepatan koneksi ...,Bagaimana kecepatan Internet dapat ditingkatka...
3,Mengapa saya secara mental sangat kesepian? Ba...,Temukan sisanya ketika [math] 23 ^ {24} [/ mat...
4,Manakah yang larut dalam air seperti gula pasi...,Ikan apa yang akan bertahan hidup di air asin?
...,...,...
404285,Ada berapa kata kunci dalam bahasa pemrograman...,Ada berapa kata kunci dalam Bahasa Pemrograman...
404286,Apakah Anda percaya ada kehidupan setelah kema...,Benarkah ada kehidupan setelah kematian?
404287,Apa satu koin?,Koin apa ini?
404288,Berapa perkiraan biaya hidup tahunan selama be...,Saya mengalami sedikit masalah rambut rontok t...


In [ ]:
#Model runtime will increase if tranable=True to about 900-1000s per epoch
embed = hub.KerasLayer("https://tfhub.dev/google/nnlm-id-dim128-with-normalization/2",input_shape=[], dtype=tf.string)
#embed.trainable=True

In [ ]:
xt1=embed(Q1_train)
xt2=embed(Q2_train)
xte1=embed(Q1_test)
xte2=embed(Q2_test)

In [ ]:
xt1

<tf.Tensor: shape=(323430, 128), dtype=float32, numpy=
array([[ 0.03792422, -0.08203722, -0.03952134, ..., -0.03751129,
        -0.13026127, -0.21294448],
       [ 0.16049062,  0.05221158, -0.03756306, ..., -0.06843793,
        -0.16083518,  0.00526717],
       [ 0.00541904,  0.03682816, -0.10739181, ..., -0.05703003,
        -0.12414219, -0.05216019],
       ...,
       [ 0.17406467,  0.06946962, -0.02178098, ..., -0.05727712,
        -0.17851247, -0.00691927],
       [ 0.14288372, -0.16400667,  0.03347619, ...,  0.04388543,
        -0.12863201, -0.02391369],
       [-0.02106932, -0.22670205, -0.11702876, ..., -0.13742013,
        -0.11424292,  0.08969257]], dtype=float32)>

# Final Models
## Models with two inputs and embedding integrated

## Model with GRU layer

In [ ]:
# Model with Bidirectional LSTM, GRU, without Attention, will run for about 150-200s per epoch
qstn1=tf.keras.Input(shape=(), dtype=tf.string)
qstn2=tf.keras.Input(shape=(), dtype=tf.string)

qu1=embed(qstn1)
qu2=embed(qstn2)
concat=tf.keras.layers.concatenate([qu1,qu2])
concat=tf.keras.layers.Reshape((2,128))(concat)
concat=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(concat)
concat=tf.keras.layers.GRU(64, return_sequences=True)(concat)
concat=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
concat=tf.keras.layers.Dense(256,activation='relu')(concat)
concat=tf.keras.layers.Dropout(0.2)(concat)
concat=tf.keras.layers.BatchNormalization()(concat)
is_dupe=tf.keras.layers.Dense(1, activation="sigmoid")(concat)

model4 = tf.keras.models.Model(inputs=[qstn1,qstn2], outputs=is_dupe)
model4.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, 128)          112461824   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 256)          0           keras_layer[0][0]            

In [ ]:
tf.keras.utils.plot_model(model4, show_shapes=True)

In [ ]:
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history4 = model4.fit([Q1_train, Q2_train],
                    yt,
                    epochs=10,
                    validation_data=([Q1_test,Q2_test], yte),
                    verbose=2)

Epoch 1/10
10108/10108 - 253s - loss: 0.5762 - accuracy: 0.6988 - val_loss: 0.5281 - val_accuracy: 0.7285
Epoch 2/10
10108/10108 - 218s - loss: 0.5016 - accuracy: 0.7464 - val_loss: 0.4768 - val_accuracy: 0.7610
Epoch 3/10
10108/10108 - 210s - loss: 0.4701 - accuracy: 0.7662 - val_loss: 0.4647 - val_accuracy: 0.7710
Epoch 4/10
10108/10108 - 211s - loss: 0.4495 - accuracy: 0.7799 - val_loss: 0.4487 - val_accuracy: 0.7809
Epoch 5/10
10108/10108 - 211s - loss: 0.4339 - accuracy: 0.7906 - val_loss: 0.4371 - val_accuracy: 0.7867
Epoch 6/10
10108/10108 - 212s - loss: 0.4195 - accuracy: 0.7997 - val_loss: 0.4341 - val_accuracy: 0.7876
Epoch 7/10
10108/10108 - 212s - loss: 0.4073 - accuracy: 0.8071 - val_loss: 0.4389 - val_accuracy: 0.7868
Epoch 8/10
10108/10108 - 211s - loss: 0.3961 - accuracy: 0.8139 - val_loss: 0.4296 - val_accuracy: 0.7960
Epoch 9/10
10108/10108 - 215s - loss: 0.3858 - accuracy: 0.8207 - val_loss: 0.4220 - val_accuracy: 0.7987
Epoch 10/10
10108/10108 - 213s - loss: 0.3761 

In [ ]:
prediction4 = model4.predict([np.asarray(["saya memukul kursi"]),np.asarray(["kursi dipukul oleh saya"])])
print(prediction4)

[[0.90333223]]


In [ ]:
model4.predict([np.asarray(["saya yang mempunyai gitar yamaha itu"]),np.asarray(["gitar yamaha itu milik saya"])])

array([[0.84960437]], dtype=float32)

In [ ]:
model4.predict([np.asarray(["saya bermain di atas meja"]),np.asarray(["kursi diduduki oleh saya"])])

array([[0.05997301]], dtype=float32)

### Save the first model

In [ ]:
export_dir1 = 'saved_model/model_gru1'
tf.saved_model.save(model4, export_dir=export_dir1)

INFO:tensorflow:Assets written to: saved_model/model_gru1/assets


INFO:tensorflow:Assets written to: saved_model/model_gru1/assets


### Try to load model from saved model

In [ ]:
new_model4 = tf.keras.models.load_model(export_dir1)
# nnew_model4 = tf.keras.models.load_model("/content/drive/MyDrive/saved_model/model_gru1")
new_model4.summary()
# nnew_model4.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        (None, 128)          112461824   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 256)          0           keras_layer[0][0]            

In [ ]:
new_model4.predict([np.asarray(["saya memukul kursi"]),np.asarray(["kursi dipukul oleh saya"])])
# nnew_model4.predict([np.asarray(["saya memukul kursi"]),np.asarray(["kursi dipukul oleh saya"])])

array([[0.90333223]], dtype=float32)

# Save model to Drive

In [ ]:
#save model to MyDrive
!cp -r /content/saved_model /content/drive/MyDrive/